# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor


In [2]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("../data/wikipedia_train3.csv",parse_dates=['date'])
test = pd.read_csv("../data/wikipedia_test3.csv",parse_dates=['date'])

In [3]:
train.shape, test.shape
train.head()

,Page,date,Visits
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-01-01,2660.0
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-01-01,9936.0
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-01-01,1712.0
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-01-01,3175.0
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-01-01,1682.0


**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

In [4]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

def smape_sfast(y_true, y_pred):
    return np.mean( (np.abs(y_pred - y_true)) / ((np.abs(y_true) + np.abs(y_pred))/2)) * 100

In [5]:
start_date = train.date.unique()[-15:][0]
start_date

numpy.datetime64('2016-08-17T00:00:00.000000000')

In [6]:
split_date = train.date.unique()[train.date.unique().size - int((test.date.unique().size/(test.date.unique().size + train.date.unique().size))*train.date.unique().size)]
split_date

numpy.datetime64('2016-07-14T00:00:00.000000000')

In [7]:
### Your code here
from dateutil.relativedelta import relativedelta
V_train, V_test = train[train.date < split_date], train[train.date >= pd.to_datetime(split_date) + relativedelta(days=10)]
Vmedian_baseline = V_train.loc[V_train.date >= V_train.date.unique()[-15]].groupby(['Page']).median(numeric_only = True).reset_index()
Vmedian_baseline.rename(columns={"Visits": "pred_Visits"}, inplace=True)

median_baseline = train.loc[train.date >= train.date.unique()[-15]].groupby(['Page']).median(numeric_only = True).reset_index()
median_baseline.rename(columns={"Visits": "pred_Visits"}, inplace=True)

In [8]:
V_test.head()

,Page,date,Visits
2422075,Special:Search_zh.wikipedia.org_all-access_spider,2016-07-24,1514.0
2422076,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-07-24,8308.0
2422077,Project:上传_zh.wikipedia.org_all-access_spider,2016-07-24,3456.0
2422078,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-07-24,4864.0
2422079,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-07-24,946.0


In [9]:
Vpred1 = V_test.merge(Vmedian_baseline, on="Page", how='left')
#Vpred1

In [10]:
pred = test.merge(median_baseline, on="Page", how='left')
#pred

In [11]:
smape_fast(Vpred1['Visits'], Vpred1['pred_Visits'])

40.42231893101511

In [12]:
smape_fast(pred['Visits'], pred['pred_Visits'])

40.00701270204617

**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [13]:
### Your code here
def create_validation(df, start_date):
    return df.loc[(df['date'] >= pd.to_datetime(start_date) - relativedelta(days = 21)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(days = 0))].index, \
           df.loc[(df['date'] >= pd.to_datetime(start_date) + relativedelta(days = 10) ) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months = 3) )].index


In [14]:
train['DayOfWeek'] = train["date"].dt.dayofweek
V_test['DayOfWeek'] = V_test["date"].dt.dayofweek
test['DayOfWeek'] = test["date"].dt.dayofweek

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
### Your validation iterator
train_dates = ['2016-01-22', '2016-02-15', '2016-04-01', '2016-05-15', '2016-06-01']
CViterator = []
for i in train_dates:
    trainIndices, valIndices = create_validation(train, i)
    CViterator.append( (trainIndices, valIndices) )

In [16]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat(
        [df[cat_cols] for df in df_list],
        axis=0
    ).fillna('').apply(
        lambda x: d[x.name].fit(x)
    ),
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(
            lambda x: d[x.name].transform(x))

In [17]:
# label encode data (categorical + binary)
__encode_categorical(df_list=[train, test, V_test], cat_cols=['Page'])


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [18]:
X_train = train.drop(['Visits', 'date'], axis = 1)
y_train = train['Visits']

X_testV = V_test.drop(['Visits', 'date'], axis = 1)
y_testV = V_test['Visits']

X_test = test.drop(['Visits','date'], axis = 1)
y_test = test['Visits']

Yeah, I know, that DT regressor it's not the best estimator for this specific case; but it seems that other estimators will be fitting forever, especially with proper features encoding.

In [19]:
DTregressor = DecisionTreeRegressor()
params = {
    'max_depth': (None, 30)
}
grid_search = GridSearchCV(DTregressor, param_grid = params, cv=CViterator, n_jobs = -1, verbose=2)

%time grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   17.3s finished


Wall time: 29.4 s


GridSearchCV(cv=[(Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            248105, 248106, 248107, 248108, 248109, 248110, 248111, 248112,
            248113, 248114],
           dtype='int64', length=248115), Int64Index([ 366265,  366266...2882855, 2882856,
            2882857, 2882858, 2882859],
           dtype='int64', length=968830))],
       error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': (None, 30)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=2)

In [20]:
grid_search.best_params_

{'max_depth': 30}

In [21]:
y_predV = grid_search.predict(V_test.drop(['Visits', 'date'], axis = 1))

In [22]:
smape_fast(V_test['Visits'], y_predV)

58.1457268281396

In [23]:
y_pred = grid_search.predict(X_test)

In [24]:
smape_fast(test['Visits'], y_pred)

60.67596186586641

## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [25]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [26]:
train2 = pd.read_csv('../data/train_french.csv')
test2 = pd.read_csv('../data/test_french.csv')

In [27]:
train2.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,62510,1,NON,500,Indivividu,agent; employe,Sans information,2,34,A,...,0,194,49,0,0,1,NON,30,2,67.68
1,83126,2,NON,500,Indivividu,cadre; ingenieur,Sans information,2,37,A,...,0,0,100,0,0,5,NON,12,1,120.60
2,68224,3,NON,500,Indivividu,enseignant; formateur; chercheur,Maison,1,1,E,...,1,98,100,0,0,5,NON,30,2,100.62
3,30263,3,NON,500,Famille,agent; employe,Maison,2,34,A,...,0,151,38,0,0,1,NON,15,1,58.73
4,72231,2,NON,500,Famille,educateur; animateur; moniteur,Appartement,1,19,D,...,0,39,100,0,0,1,NON,0,2,132.65


In [28]:
features = train2.columns.drop('prime_tot_ttc')
uniqueValues = pd.Series([train2[col].value_counts().size for col in features] , index = features)

In [29]:
binary_cols = [
                 'veranda',
                 'sit_fam',
                 'societe',
                 'objets_precieux',
                 'pres_contrat_auto',
                 'formule',
                 'var3',
                 'var6',
                 'var7',
                 'var9',
                 'var11'
                ]
categorical_cols = ['nature_lri', 'niv_urbanisation', 'profession', 'code_postal']
ordinal_cols = [
                 'nb_pieces',
                 'biens_nomades',
                 'domaine_activite',
                 'age_societaire',
                 'sit_juridique',
                 'surf_habitat',
                 'patrimoine_mob',
                 'var1',
                 'var2',
                 'var4',
                 'var5',
                 'var8',
                 'var10'
                ]

In [30]:
__encode_categorical(df_list = [train2, test2], cat_cols = categorical_cols + binary_cols)

In [31]:
train2.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,16131,1,0,500,1,1,5,1,34,0,...,0,194,49,0,0,1,0,30,1,67.68
1,22013,2,0,500,1,4,5,1,37,0,...,0,0,100,0,0,5,0,12,0,120.60
2,18092,3,0,500,1,10,2,0,1,4,...,1,98,100,0,0,5,0,30,1,100.62
3,7493,3,0,500,0,1,2,1,34,0,...,0,151,38,0,0,1,0,15,0,58.73
4,19264,2,0,500,0,9,0,0,19,3,...,0,39,100,0,0,1,0,0,1,132.65


In [32]:
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(train2.drop('prime_tot_ttc', axis = 1), 
                                                    train2['prime_tot_ttc'],
                                                    test_size = test2.shape[0] / (train2.shape[0] + test2.shape[0]),
                                                    random_state = 0 
                                                    )

In [33]:
DTregressor2 = DecisionTreeRegressor()
params = {
    'max_depth': (None, 30)
}
grid_search2 = GridSearchCV(DTregressor2, param_grid = params, cv = 3, n_jobs = -1, verbose=2)

%time grid_search2.fit(X_train2, y_train2)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:   14.0s remaining:   14.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   16.7s finished


Wall time: 19.7 s


GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': (None, 30)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=2)

In [34]:
mape(y_test2, grid_search2.predict(X_test2))

11.292139885291572

In [35]:
mape(test2['prime_tot_ttc'], grid_search2.predict(test2.drop('prime_tot_ttc', axis = 1)))

11.67825645753657